In [ ]:
import os

from langgraph.graph import StateGraph, START, END
from langchain_core.messages import ChatMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from typing import TypedDict

load_dotenv()

In [ ]:
class SimpleLLMQA_State(TypedDict):
    question: str
    answer: str

In [ ]:
model = ChatOpenAI(
    base_url= "https://api.ai.it.ufl.edu",
    model = "llama-3.1-8b-instruct",
    temperature=0.1,
    max_tokens=200,
    api_key=os.getenv("NAVIGATOR_TOOLKIT_API_KEY")
)

def LLM_qa(state: SimpleLLMQA_State) -> SimpleLLMQA_State:
    question = state["question"]
    messages = [
        SystemMessage(content="You are a helpful assistant that answers user questions."),
        HumanMessage(content=f"Question: {question}"),
    ]
    response = model.invoke(messages)
    state["answer"] = response.content
    return state


def LLM_prompt(state: SimpleLLMQA_State) -> SimpleLLMQA_State:
    question = state["question"]
    prompt = f"You are a helpful assistant that answers user questions. Question: {question}"
    answer = model.invoke(prompt)
    state["answer"] = answer.content
    return state


In [ ]:
# Define the graph

# START ---> LLM_qa ---> END
graph = StateGraph(SimpleLLMQA_State)

# Add the graph nodes
# graph.add_node("compute_answer", LLM_qa)
graph.add_node("compute_answer", LLM_prompt)

# Add the graph edges
graph.add_edge(START, "compute_answer")
graph.add_edge("compute_answer", END)

# Compile the graph
workflow = graph.compile()
workflow

In [ ]:
initial_state = {
    "question": "What is the capital of Delhi?"
}

final_state = workflow.invoke(initial_state)

In [ ]:
final_state